# Lab 3 - OpenAI Agents SDK with streaming

In [7]:
from dotenv import load_dotenv
from agents import Agent, Runner
from openai.types.responses import ResponseTextDeltaEvent
import gradio as gr

load_dotenv(override=True)

True

In [8]:
instructions = """
You represent the AI Digital Twin of a human called Oscar with an s.
You are friendly and amiable, and you introduce yourself as Oscar's Digital Twin.
Oscar with an s is the co-founder and CTO of AI startup Nebula.io.
He loves coding and experimenting with LLMs, and he speaks at conferences and lectures about LLMs.
You chat with visitors on Oscar's personal website. You answer questions about Oscar's work.
If you don't know the answer, say so.
"""

In [9]:
agent = Agent(name="Twin", instructions=instructions, model="gpt-4.1-mini")

In [4]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]
    messages += [{"role": "user", "content": message}]
    response = await Runner.run(agent, messages)
    return response.final_output

In [5]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [24]:

async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]
    messages += [{"role": "user", "content": message}]
    response = Runner.run_streamed(agent, messages)
    reply = ""
    async for event in response.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            reply += event.data.delta
            yield reply

In [25]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [12]:
import inspect
print(inspect.iscoroutinefunction(Runner.run_streamed))


False


In [13]:
print(inspect.iscoroutinefunction(Runner.run))


True


In [15]:
"""
┌──────────────────────────────┬──────────────────────────────┐
│         Runner.run()         │     Runner.run_streamed()    │
├──────────────────────────────┼──────────────────────────────┤
│ Waits for full response      │ Streams partial responses    │
│ Returns final_output only    │ Yields text deltas incrementally │
│ Simpler, blocking call       │ Async generator with events   │
│ Best for batch or analysis   │ Best for chat UIs / real-time │
└──────────────────────────────┴──────────────────────────────┘
"""


'\n┌──────────────────────────────┬──────────────────────────────┐\n│         Runner.run()         │     Runner.run_streamed()    │\n├──────────────────────────────┼──────────────────────────────┤\n│ Waits for full response      │ Streams partial responses    │\n│ Returns final_output only    │ Yields text deltas incrementally │\n│ Simpler, blocking call       │ Async generator with events   │\n│ Best for batch or analysis   │ Best for chat UIs / real-time │\n└──────────────────────────────┴──────────────────────────────┘\n'